In [2]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt   
import seaborn as sns
from dotenv import load_dotenv
import os
import numpy as np
load_dotenv()
db_url = os.getenv("DB_URL")

engine = create_engine(db_url)



In [3]:
import numpy as np 
query="""show tables 
"""
df=pd.read_sql(query,engine)

df.head(20)



,Tables_in_olist_dw
0,dim_customers
1,dim_orders
2,dim_products
3,dim_sellers
4,fact_order_items
5,fact_order_payments
6,fact_order_reviews
7,staging_customers
8,staging_geolocation
9,staging_order_items


In [4]:
df_orders = pd.read_sql(
    "SELECT * FROM dim_orders",
    engine
) 
df_order_items = pd.read_sql(
    "SELECT * FROM fact_order_items",
    engine
) 
df_order_payments = pd.read_sql(
    "SELECT * FROM fact_order_payments",
    engine
) 
df_order_reviews = pd.read_sql(
    "SELECT * FROM fact_order_reviews",
    engine
) 
df_products=pd.read_sql(
    "SELECT * FROM dim_products",
    engine
)
df_customers=pd.read_sql(
    "select * from dim_customers",
    engine)
	
df_sellers=pd.read_sql(
    "select * from dim_sellers",
    engine)


In [5]:
#df_orders.iloc[:,[1,2]]
df_orders=df_orders[['order_id','customer_id','order_purchase_timestamp','order_delivered_customer_date']]
df_order_items.rename(columns={'freight_value':'delivery_cost'},inplace=True)


#review table 
df_order_reviews=df_order_reviews[['order_id','review_score','review_creation_date']]

fact=pd.merge(df_orders,df_order_items,on='order_id')


fact.drop('shipping_limit_date',axis=1,inplace=True)
fact.head()


,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [6]:
fact=pd.merge(fact,df_order_reviews,on='order_id')
fact.head()

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,5,2017-09-21 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,4,2017-05-13 00:00:00
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,5,2018-01-23 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,4,2018-08-15 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,5,2017-03-02 00:00:00


In [7]:
fact.describe()
#missing=fact['order_delivered_customer_date'].isna().sum()
#print(missing)
fact[fact['order_delivered_customer_date'].isna()]
fact['delivery_time']=fact['order_delivered_customer_date']-fact['order_purchase_timestamp']
#fact.head()
#(fact['order_delivered_customer_date'] - fact['order_purchase_timestamp']).isna().sum()


fact['delivery_time']=fact['delivery_time'].dt.total_seconds()/(86400)
fact['delivery_time']=(fact['delivery_time'].round(2))
fact.head()


,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date,delivery_time
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,5,2017-09-21 00:00:00,7.61
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,4,2017-05-13 00:00:00,16.22
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,5,2018-01-23 00:00:00,7.95
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,4,2018-08-15 00:00:00,6.15
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,5,2017-03-02 00:00:00,25.11


TRANSFORM LOAD 

In [8]:
#Grain Check 
fact.groupby("order_id")["order_item_id"].nunique().describe()
fact.groupby("order_id")["review_score"].nunique().describe()


count    97441.000000
mean         1.000759
std          0.027548
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: review_score, dtype: float64

In [9]:
#(fact['delivery time']>150).sum()
print(fact['delivery_time']>150)
#fact['delivery time'].describe()
fact['delivery_time']>150
fact_super_late = fact[fact['delivery_time'] > 150]
fact_super_late.head()


0         False
1         False
2         False
3         False
4         False
          ...  
111455    False
111456    False
111457    False
111458    False
111459    False
Name: delivery_time, Length: 111460, dtype: bool


,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date,delivery_time
1315,031e7d4e559a1bf08e71a419aa998d0a,aedfc62ab52a3e4ea642220ccdec91ff,2017-04-06 12:59:46,2017-09-19 16:23:52,1,fb1f078a066428f00c1d71d6b9df3ff8,eebb3372362aa9a46975164bed19a7e7,73.99,24.52,1,2017-05-13 00:00:00,166.14
6687,0f4519c5f1c541ddec9f21b3bddd533a,1a8a4a30dc296976717f44e7801fdeef,2017-03-09 13:26:57,2017-09-19 14:38:21,1,e0d64dcfaa3b6db5c54ca298ae101d05,7e93a43ef30c4f03f38b393420bc753a,231.27,27.88,4,2017-03-29 00:00:00,194.05
11882,1b3190b2dfa9d789e1f14c05b647a14a,d306426abe5fca15e54b645e4462dc7b,2018-02-23 14:57:35,2018-09-19 23:24:07,1,ee406bf28024d97771c4b1e8b7e8e219,7a67c85e85bb2ce8582c35f2203ad736,144.99,17.26,2,2018-03-17 00:00:00,208.35
17433,285ab9426d6982034523a855f55a885e,9cf2c3fa2632cee748e1a59ca9d09b21,2017-03-08 22:47:40,2017-09-19 14:00:04,1,0c6fc9b9317a68d1cda098c063914b72,c847e075301870dd144a116762eaff9a,429.90,27.75,1,2017-04-08 00:00:00,194.63
18809,2ba1366baecad3c3536f27546d129017,6852966131028b6696be63d0a1bd87ae,2017-02-28 14:56:37,2017-08-28 16:23:46,1,e4176515d2055eb7771645c597f8b40c,8b28d096634035667e8263d57ba3368c,34.90,17.78,1,2017-04-01 00:00:00,181.06


In [10]:
type(fact)

pandas.core.frame.DataFrame

In [ ]:
fact[fact['delivery_time'] < 1]

fact['has_review'] = fact['review_score'].notna()

fact['super_late'] = fact['delivery_time'] > 120



fact=fact.drop(columns=['has_review'])

fact.head()

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date,delivery_time,super_late
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,5,2017-09-21 00:00:00,7.61,False
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,4,2017-05-13 00:00:00,16.22,False
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,5,2018-01-23 00:00:00,7.95,False
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,4,2018-08-15 00:00:00,6.15,False
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,5,2017-03-02 00:00:00,25.11,False


In [12]:
#late_fact=fact[fact['super_late']] 
#late_fact.head()
#fact.groupby('super_late')['review_score'].mean()
late_review_avg = (
    fact.groupby('super_late')['review_score']
        .mean()
        .reset_index(name='avg_review_score')
)

late_review_avg['super_late']=late_review_avg['super_late'].map({True:'more than 150',False:'less than 150' })
late_review_avg


,super_late,avg_review_score
0,less than 150,4.034978
1,more than 150,3.043478


In [29]:
fact['high_price']=fact['price']>fact['price'].mean()
fact[fact['high_price']==False]
fact['high_delivery_cost']=fact['delivery_cost']>fact['delivery_cost'].mean()
fact.groupby('high_price')['review_score'].mean()
fact

,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_item_id,product_id,seller_id,price,delivery_cost,review_score,review_creation_date,delivery_time,super_late,high_price,high_delivery,high_delivery_cost
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-20 23:43:48,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,5,2017-09-21 00:00:00,7.61,False,False,False,False
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-05-12 16:04:24,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,4,2017-05-13 00:00:00,16.22,False,True,False,False
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-22 13:19:16,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,5,2018-01-23 00:00:00,7.95,False,True,False,False
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-14 13:32:39,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,4,2018-08-15 00:00:00,6.15,False,False,False,False
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-03-01 16:42:31,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,5,2017-03-02 00:00:00,25.11,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111455,fffc94f6ce00a00581880bf54a75a037,b51593916b4b8e0d6f66f2ae24f2673d,2018-04-23 13:57:06,2018-05-10 22:56:40,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,299.99,43.41,5,2018-05-11 00:00:00,17.37,False,True,True,True
111456,fffcd46ef2263f404302a634eb57f7eb,84c5d4fbaf120aae381fad077416eaa0,2018-07-14 10:26:46,2018-07-23 20:31:55,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,350.00,36.53,5,2018-07-24 00:00:00,9.42,False,True,True,True
111457,fffce4705a9662cd70adb13d4a31832d,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56,2017-10-28 12:22:22,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,99.90,16.95,5,2017-10-29 00:00:00,4.80,False,False,False,False
111458,fffe18544ffabc95dfada21779c9644f,b5e6afd5a41800fdf401e0272ca74655,2017-08-14 23:02:59,2017-08-16 21:59:40,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,55.99,8.72,5,2017-08-17 00:00:00,1.96,False,False,False,False


% of orders delivered late


In [ ]:
late_pct = fact['super_late'].mean() * 100
on_time_pct = 100 - late_pct
pd.DataFrame({
    'delivery_status': ['Late', 'On-time'],
    'percentage': [late_pct, on_time_pct]
})


99.95872958909025


,delivery_status,percentage
0,Late,0.04127
1,On-time,99.95873


In [13]:

'''
plt.figure(figsize=(7,5))
plt.scatter(fact['delivery_cost'], fact['delivery_time'], alpha=0.3)
\plt.xlabel('Delivery Cost')
plt.ylabel('Delivery Time (days)')
plt.title('Delivery Cost vs Delivery Time')
plt.show()
'''


"\nplt.figure(figsize=(7,5))\nplt.scatter(fact['delivery_cost'], fact['delivery_time'], alpha=0.3)\n\\plt.xlabel('Delivery Cost')\nplt.ylabel('Delivery Time (days)')\nplt.title('Delivery Cost vs Delivery Time')\nplt.show()\n"

In [14]:


#df_order_items.groupby(['seller_id'])['price'].count()
#df_order_items[df_order_items['seller_id'].isin(df_order_items.nlargest(5, 'price')['seller_id'])]
#df_order_items(['price'])
df_order_items[df_order_items['price']<=6000].sort_values(['price'],ascending=False).head(5)
#df_order_items.nlargest(5,'price')


#df_order_items.plot(kind='scatter',x='price',y='seller_id')

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,delivery_cost
74336,a96610ab360d42a2e5335a3998b4718a,1,a6492cc69376c469ab6f61d8f44de961,59417c56835dd8e2e72f91f809cd4092,2017-04-18 13:25:18,4799.00,151.34
11249,199af31afc78c699f0dbf71fb178d4d4,1,c3ed642d592594bb648ff4a04cee2747,59417c56835dd8e2e72f91f809cd4092,2017-05-09 15:50:15,4690.00,74.34
62086,8dbc85d1447242f3b127dda390d56e19,1,259037a6a41845e455183f89c5035f18,c72de06d72748d1a0dfb2125be43ba63,2018-06-28 12:36:36,4590.00,91.78
29193,426a9742b533fc6fed17d1fd6d143d7e,1,a1beef8f3992dbd4cd8726796aa69c53,512d298ac2a96d1931b6bd30aa21f61d,2018-08-16 14:24:28,4399.87,113.45
45843,68101694e5c5dc7330c91e1bbc36214f,1,6cdf8fc1d741c76586d8b6b15e9eef30,ed4acab38528488b65a9a9c603ff024a,2018-04-05 08:27:27,4099.99,75.27
